In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#laad data
with open('input.txt') as t:
    text = t.read()

**Deel 1**

In [3]:
# maak een nieuw bestand met per regel opeenvogende waarden en een enter bij een witregel
new = ''
for line in text.split('\n'):
    # lege regel: voeg een enter toe
    if line == '':
        new += '\n'
    # voeg een spatie aan het einde toe voor het plakken van waarden die van verschillende regels komen
    else:
        new += line + ' ' 
# maak een lijst, een item is een paspoort (string)
textlist = list(new.split('\n'))

In [4]:
lwb = []
for line in textlist:
    items = line.split()
    # maak een dictonary per paspoort
    wb = {}
    for item in items:        
        key, val = item.split(':')
        wb[key] = val
    #maak een lijst van deze dictonaries
    lwb.append(wb.copy())
# maak een dataframe met de categorieen als kolommen en de paspoorten als rijen
df = pd.DataFrame(lwb)

In [5]:
# verwijder de optionele 'cid' kolom en verwijder de rijen met missende waarden in de overige kolommen
df_valid = df.drop("cid", 1).dropna()
# tel het aantal geldige paspoorten
print(f'Aantal geldige paspoorten: {len(df_valid.index)}')

Aantal geldige paspoorten: 264


**Deel 2**

    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

In [6]:
# defineer functie voor de jaartal condities
df_check = df_valid.copy()
def test_rules(column, length, minv, maxv):
    li = []
    for item in column:
        rules = [len(item) == length, pd.to_numeric(item) >= minv, pd.to_numeric(item) <= maxv]
        if all(rules):
            li.append(True)
        else: 
            li.append(False)
    return(li)

In [7]:
# pas functie toe voor de jaartal variabelen
df_check['byr'] = test_rules(df_valid['byr'], 4, 1920, 2002)
df_check['iyr'] = test_rules(df_valid['iyr'], 4, 2010, 2020)
df_check['eyr'] = test_rules(df_valid['eyr'], 4, 2020, 2030)

In [8]:
# hgt (Height)
r = []
for item in df_valid['hgt']:
    s = re.split('(cm|in)', item)
    if len(s) >= 2:
        if s[1] == 'cm':
            if ((int(s[0]) >= 150) & (int(s[0]) <= 193)):
                r.append(True)
            else: 
                r.append(False)
        elif s[1] == 'in':
            if ((int(s[0]) >= 59) & (int(s[0]) <= 76)):
                r.append(True)
            else: 
                r.append(False)
        else:
                r.append(False)
    else:
            r.append(False)
df_check['hgt'] = r

In [9]:
# hcl (Hair Color)
hcl_re = r'(^#[0-9a-z]{6}$)'
df_check['hcl'] = pd.isna(df_valid['hcl'].str.extract(hcl_re)) == False

In [10]:
# ecl (Eye Color)
ecl_re = r'(amb|blu|brn|gry|grn|hzl|oth)'
df_check['ecl'] = pd.isna(df_valid['ecl'].str.extract(ecl_re)) == False

In [11]:
# pid (Passport ID)
pid_re = r'(^\d{9}$)'
df_check['pid'] = pd.isna(df_valid['pid'].str.extract(pid_re)) == False

In [12]:
# tel rijen waar alle waarden 'True' zijn (true = 1, dus gemiddelde is dan 1)
print(f'Aantal geldige paspoorten: {len(df_check.loc[df_check.mean(axis = 1) == 1,:])}')


Aantal geldige paspoorten: 224
